In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 13 15:44:37 2022

@author: jeffr
"""

'''
DESCRIPTION: applies chosen augmentation method to supplied dataframe.
It is assumed that that the labels column is located at the END of the dataframe.

INPUTS:
data = dataframe
nrows = number of rows that will be created in augmentation

OPTIONAL:
nvalues = number of values being swapped, limited to column length

PMONE and GAUSNOISE ONLY:
unit = amount being plus or minused
noise = 
'''

In [43]:
#import libraries
import numpy as np
import pandas as pd
import random

In [45]:
def betterApplyAugmentationMethods(data, method, nrows, nvalues=None, unit=None, noise=None):
    
    # If nvalues not specified, entire column is selected
    if nvalues == None:
        nvalues = data.shape[1]-1
        
    #FABI CODE
    # Assumes the last column is the label column
    label_column = data.columns[-1]  # Assumes the last column is the label column
    #FABI CODE STOPS HERE
    
    if str(method).lower() == 'gausnoise':
        # Creates empty dataframe to hold augmented rows
        augmentedDf = pd.DataFrame()
        
        # Selects random rows from data and appends to augmentedDf
        for i in range(nrows):
            augmentedDf = pd.concat([augmentedDf, data.iloc[[random.randint(0, data.shape[0]-1)]]], ignore_index=True)
        
        # Drops label column of augmentedDf
        #augmentedDf = augmentedDf.drop(augmentedDf.shape[1]-1, axis=1)

        #FABI CODE
        augmentedDf = augmentedDf.drop(label_column, axis=1) 
        #FABI CODE STOPS HERE
        
        # Selects random unique column index 
        randCols = random.sample(range(0, data.shape[1]-1), nvalues)
        
        # Applies Gaussian noise to randCols values stored in array
        for i in range(augmentedDf.shape[0]):
            for cols in randCols:
                augmentedDf.iloc[i, cols] += np.random.normal(0, noise)
            
        
        # Combines both data and augmentedDf for full augmented dataframe
        augmentedDf = pd.concat([data, augmentedDf], ignore_index=True)
            
        
        return augmentedDf
    
    elif str(method).lower() == 'modgausnoise':
        # Creates an empty dataframe to hold augmented observations
        augmentedDf = pd.DataFrame()
        
        # Randomly selects unique column indexs from data
        randCols = random.sample(range(0, data.shape[1]-1), nvalues)
        
        # Appends randomly selected rows from data to augmentedDf
        for i in range(nrows):
            augmentedDf = pd.concat([augmentedDf, data.iloc[[random.randint(0, data.shape[0]-1)]]], ignore_index=True)
            
        # Drops labels from augmentedDf
        #augmentedDf = augmentedDf.drop(augmentedDf.shape[1]-1, axis=1)
        #FABI CODE
        augmentedDf = augmentedDf.drop(label_column, axis=1) 
        #FABI CODE STOPS HERE
        
        # Generates Gaussian distribution based on columns summary statistics
        # Swaps value with random value in generated Gaussian distribution
        for col in randCols:
            for i in range(augmentedDf.shape[0]):
                mean = augmentedDf[col].mean()
                stDev = augmentedDf[col].std()
                
                augmentedDf.iloc[i, col] =  np.random.normal(mean, stDev)
            
        # Combines both data and augmentedDf into final dataframe
        #FABI CODE
        # Add back label column with NaN values
        augmentedDf[label_column] = np.nan  
        #FABI CODE STOPS HERE
        augmentedDf = pd.concat([data, augmentedDf], axis=0, ignore_index=True)

        return augmentedDf
    
    else:
        print("Method not found")
        return None

In [47]:
#This is where you begin coding!
# Load the dataset
df = pd.read_csv('IRIS.csv')

# Define parameters for augmentation

nrows = 10  # Number of new rows to be created
nvalues = 4  # Number of columns to apply noise to (adjust as needed)
noise = 0.1  # Standard deviation of Gaussian noise

# Apply the gausnoise method
augmented_df_gausnoise = betterApplyAugmentationMethods(df, 'gausnoise', nrows, nvalues, noise=noise)
print("Gausnoise Method:")
print(augmented_df_gausnoise)

# Apply the modgausnoise method
augmented_df_modgausnoise = betterApplyAugmentationMethods(df, 'modgausnoise', nrows, nvalues, noise=noise)
print("\nModGausnoise Method:")
print(augmented_df_modgausnoise)


Gausnoise Method:
     sepal_length  sepal_width  petal_length  petal_width      species
0        5.100000     3.500000      1.400000     0.200000  Iris-setosa
1        4.900000     3.000000      1.400000     0.200000  Iris-setosa
2        4.700000     3.200000      1.300000     0.200000  Iris-setosa
3        4.600000     3.100000      1.500000     0.200000  Iris-setosa
4        5.000000     3.600000      1.400000     0.200000  Iris-setosa
..            ...          ...           ...          ...          ...
155      5.607814     2.385102      3.735363     1.026157          NaN
156      7.708233     3.908343      6.758682     2.164014          NaN
157      6.337763     2.310909      4.958704     1.610042          NaN
158      6.098364     2.803600      4.841438     1.188450          NaN
159      7.472477     2.973846      6.208038     1.799129          NaN

[160 rows x 5 columns]


KeyError: 0